# Imports and Data File

In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

sys.path.insert(0, '/workspaces/detecting-dine-out-patterns/src')
from util.distance import (
    average_distance_between_rows,
    median_distance_between_rows,
    max_distance_between_rows,
    timestamps_of_max_distance,
    longest_stretch_not_moving,
    get_distance_between_rows
)

In [ ]:
file_path = '../../data/movements.csv'
movements_data = pd.read_csv(file_path, parse_dates=['datetime'])

# Distance Analysis

In [ ]:
"""
This block performs distance analysis on movement data grouped by 'id'.
"""

results = []
grouped = movements_data.groupby('id')

for name, group in grouped:
    avg_distance = average_distance_between_rows(group)
    med_distance = median_distance_between_rows(group)
    max_distance = max_distance_between_rows(group)
    timestamp1, timestamp2 = timestamps_of_max_distance(group)
    not_moving_time1, not_moving_time2 = longest_stretch_not_moving(group, 1)

    results.append({
        'id': name, 
        'avg_distance': avg_distance, 
        'med_distance': med_distance, 
        'max_distance': max_distance, 
        'max_distance time1': timestamp1, 
        'max_distance time2': timestamp2,
        'not moving time 1': not_moving_time1,
        'not moving time 2': not_moving_time2
    })

results_df = pd.DataFrame(results)
results_df.head(10)


In [ ]:
results = pd.DataFrame()
grouped = movements_data.groupby('id')

for name, group in grouped:
    distances = (get_distance_between_rows(group)*1000).round(0)
    results[name] = pd.Series(distances)

results.head(10)


In [ ]:
summary_statistics = pd.DataFrame()
pd.Series(results.apply(lambda x: np.mean(x)))

In [ ]:
results.apply(lambda x: np.nanmedian(x))

In [ ]:
results.apply(lambda x: np.max(x))